In [1]:
import tensorflow as tf
# tf.enable_eager_execution()
import numpy as np
import datetime
import os
import argparse
import matplotlib.pyplot as plt
import pandas as pd
import json
import glob
import tqdm
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers, models
from utils import *
tf.__version__

'1.15.0'

In [3]:
data_info = {
   "train_unlabel": 0, 
    "train_label": 0, 
    "validation": 0, 
    "test": 0
}
labels = ['DoS', 'Fuzzy', 'gear', 'RPM', 'Normal']
unknown_attack = ''
num_normal = 0
num_attack = 0
data_dir = './Data/Train_0.3_Labeled_0.1/'
for f in ['{}/{}/datainfo.txt'.format(data_dir, l) for l in labels if l is not unknown_attack]:
    data_read = json.load(open(f))
    for key in data_info.keys():
        data_info[key] += data_read[key]
    if 'Normal' in f:
        num_normal += data_read['train_label']
    else:
        num_attack += data_read['train_label']
        
print('Data info: ', data_info)
print('Num labeled normal: ', num_normal)
print('Num labeled attack: ', num_attack)

Data info:  {'train_unlabel': 281262, 'train_label': 31249, 'validation': 96632, 'test': 118490}
Num labeled normal:  24693
Num labeled attack:  6556


In [4]:
train_unlabel_paths = ['{}/{}/train_unlabel'.format(data_dir, l) for l in labels if l is not unknown_attack]
train_label_paths = ['{}/{}/train_label'.format(data_dir, l) for l in labels if l is not unknown_attack]
val_paths = ['{}/{}/val'.format(data_dir, l) for l in labels if l is not unknown_attack]
test_paths = ['{}/{}/test'.format(data_dir, l) for l in labels if l is not unknown_attack]

train_label = data_from_tfrecord(train_label_paths, data_info['train_label'], 1)
validation = data_from_tfrecord(val_paths,  data_info['validation'], 1)
test = data_from_tfrecord(test_paths, data_info['test'], 1)

print('Train unlabel: ', train_label_paths)
print('Train label: ', train_label_paths)
print('Validation: ', val_paths)

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
Train unlabel:  ['./Data/Train_0.3_Labeled_0.1//DoS/train_label', './Data/Train_0.3_Labeled_0.1//Fuzzy/train_label', './Data/Train_0.3_Labeled_0.1//gear/train_label', './Data/Train_0.3_Labeled_0.1//RPM/train_label', './Data/Train_0.3_Labeled_0.1//Normal/train_label']
Train label:  ['./Data/Train_0.3_Labeled_0.1//DoS/train_label', './Data/Train_0.3_Labeled_0.1//Fuzzy/train_label', './Data/Train_0.3_Labeled_0.1//gear/train_label', './Data/Train_0.3_Labeled_0.1//RPM/train_label', './Data/Train_0.3_Labeled_0.1//Normal/train_label']
Validation:  ['./Data/Train_0.3_Labeled_0.1//DoS/val', './Data/Train_0.3_Labeled_0.1//Fuzzy/val', './Data/Train_0.3_Labeled_0.1//gear/val', './Data/Train_0.3_Labeled_0.1//RPM/val', './Data/Train_0.3_Labeled_0.1//Nor

In [5]:
init = tf.global_variables_initializer()
with tf.Session(config=tf.ConfigProto(device_count={'GPU':0})) as sess:
    X_train, y_train = data_stream(train_label, sess)
    X_val, y_val = data_stream(validation, sess)
    X_test, y_test = data_stream(test, sess)
    y_train = np.argmax(y_train, axis=1)
    y_val = np.argmax(y_val, axis=1)
    y_test = np.argmax(y_test, axis=1)
    print('Train: ', X_train.shape, y_train.shape)
    print('Validation: ', X_val.shape, y_train.shape)
    print('Test: ', X_test.shape, y_test.shape)

Train:  (31249, 841) (31249,)
Validation:  (96632, 841) (31249,)
Test:  (118490, 841) (118490,)


## SVM

In [13]:
from sklearn.svm import SVC

svm = SVC()
svm.fit(X_train, y_train)

SVC()

In [15]:
y_pred = svm.predict(X_test)
evaluate(y_test, y_pred)

79370 2972
7 36141
False negative rate:  0.03609336669014598
Error rate:  0.025141362140265
Precision:  0.9999118132456505
Recall:  0.963906633309854
F1 score:  0.9815791589114452


## Decision Tree

In [16]:
from sklearn.tree import DecisionTreeClassifier
dst = DecisionTreeClassifier(random_state=0)
dst.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

In [17]:
y_pred = dst.predict(X_test)
evaluate(y_test, y_pred)

66827 15515
1112 35036
False negative rate:  0.18842146170848412
Error rate:  0.14032407798126423
Precision:  0.9836323761020915
Recall:  0.8115785382915159
F1 score:  0.8893605978134296


## ANN

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(1000, input_dim=841, activation='relu'),
  tf.keras.layers.Dense(1000, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [9]:
model.compile(optimizer='adam',
              loss= 'binary_crossentropy',
              metrics=['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
model.fit(X_train, y_train, epochs=200, batch_size=100, validation_data=(X_val, y_val))

Train on 31249 samples, validate on 96632 samples
Epoch 1/200
31249/31249 [==============================] - 2s 67us/sample - loss: 0.0947 - acc: 0.9674 - val_loss: 0.1805 - val_acc: 0.9424
Epoch 2/200
31249/31249 [==============================] - 2s 59us/sample - loss: 0.0273 - acc: 0.9916 - val_loss: 0.0428 - val_acc: 0.9874
Epoch 3/200
31249/31249 [==============================] - 2s 59us/sample - loss: 0.0143 - acc: 0.9957 - val_loss: 0.0720 - val_acc: 0.9806
Epoch 4/200
31249/31249 [==============================] - 2s 59us/sample - loss: 0.0101 - acc: 0.9963 - val_loss: 0.0971 - val_acc: 0.9765
Epoch 5/200
31249/31249 [==============================] - 2s 61us/sample - loss: 0.0039 - acc: 0.9988 - val_loss: 0.0748 - val_acc: 0.9851
Epoch 6/200
31249/31249 [==============================] - 2s 61us/sample - loss: 0.0107 - acc: 0.9964 - val_loss: 0.0875 - val_acc: 0.9809
Epoch 7/200
31249/31249 [==============================] - 2s 60us/sample - loss: 0.0048 - acc: 0.9985 - val_l

In [11]:
y_pred = model.predict(X_test).flatten().astype(int)
evaluate(y_test, y_pred)

74169 8173
0 36148
False negative rate:  0.09925675839790143
Error rate:  0.06897628491855853
Precision:  1.0
Recall:  0.9007432416020986
F1 score:  0.9477800282408265
